# ETL

(Falta diagrama)

## Extraer

Para 4 dígitos

In [93]:
import os
import requests
import PyPDF2

# Base URL of the webpage to scrape for PDFs
base_url_pdf = "https://appalertaamber.fgr.org.mx/Alerta/CreaAlertaPDFPublico?numero_reporte="

# Create folders if they don't exist
data_folder = "data"
pdf_folder = os.path.join(data_folder, "pdf")

for folder in [data_folder, pdf_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Generate links by changing the last 4 digits
links = [base_url_pdf + str(i).zfill(4) for i in range(1, 10000)]

# Iterate over the links and send requests to retrieve PDFs
for index, pdf_link in enumerate(links, start=1):
    # Sending request for PDF link
    response_pdf = requests.get(pdf_link)
    if response_pdf.status_code == 200:
        # Save the PDF to the pdf folder
        pdf_path = os.path.join(pdf_folder, f"A_{index}.pdf")
        with open(pdf_path, "wb") as pdf_file:
            pdf_file.write(response_pdf.content)
        print("Downloaded PDF:", pdf_path)
        
        # Read the content of the PDF
        try:
            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                text = ""
                for page_num in range(pdf_reader.numPages):
                    text += pdf_reader.getPage(page_num).extractText()
                
                # Check if the string "Publicación no disponible" is present in the text
                if "Publicación no disponible" in text:
                    os.remove(pdf_path)  # Delete the PDF file
                    print("Deleted PDF:", pdf_path)
        except PyPDF2.utils.PdfReadError:
            print("Failed to read PDF:", pdf_path)
    else:
        print("Failed to retrieve data for PDF link:", pdf_link)


Downloaded PDF: data/pdf/A_1.pdf
Deleted PDF: data/pdf/A_1.pdf
Downloaded PDF: data/pdf/A_2.pdf
Deleted PDF: data/pdf/A_2.pdf
Downloaded PDF: data/pdf/A_3.pdf
Deleted PDF: data/pdf/A_3.pdf
Downloaded PDF: data/pdf/A_4.pdf
Deleted PDF: data/pdf/A_4.pdf
Downloaded PDF: data/pdf/A_5.pdf
Deleted PDF: data/pdf/A_5.pdf
Downloaded PDF: data/pdf/A_6.pdf
Deleted PDF: data/pdf/A_6.pdf
Downloaded PDF: data/pdf/A_7.pdf
Deleted PDF: data/pdf/A_7.pdf
Downloaded PDF: data/pdf/A_8.pdf
Deleted PDF: data/pdf/A_8.pdf
Downloaded PDF: data/pdf/A_9.pdf
Deleted PDF: data/pdf/A_9.pdf
Downloaded PDF: data/pdf/A_10.pdf
Deleted PDF: data/pdf/A_10.pdf
Downloaded PDF: data/pdf/A_11.pdf
Deleted PDF: data/pdf/A_11.pdf
Downloaded PDF: data/pdf/A_12.pdf
Deleted PDF: data/pdf/A_12.pdf
Downloaded PDF: data/pdf/A_13.pdf
Deleted PDF: data/pdf/A_13.pdf
Downloaded PDF: data/pdf/A_14.pdf
Deleted PDF: data/pdf/A_14.pdf
Downloaded PDF: data/pdf/A_15.pdf
Deleted PDF: data/pdf/A_15.pdf
Downloaded PDF: data/pdf/A_16.pdf
Deleted P

Para 3 dígitos

In [94]:
import os
import requests
import PyPDF2

# Base URL of the webpage to scrape for PDFs
base_url_pdf = "https://appalertaamber.fgr.org.mx/Alerta/CreaAlertaPDFPublico?numero_reporte="

# Create folders if they don't exist
data_folder = "data"
pdf_folder = os.path.join(data_folder, "pdf")

for folder in [data_folder, pdf_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Generate links by changing the last 4 digits
links = [base_url_pdf + str(i).zfill(3) for i in range(1, 1000)]

# Iterate over the links and send requests to retrieve PDFs
for index, pdf_link in enumerate(links, start=1):
    # Sending request for PDF link
    response_pdf = requests.get(pdf_link)
    if response_pdf.status_code == 200:
        # Save the PDF to the pdf folder
        pdf_path = os.path.join(pdf_folder, f"B_{index}.pdf")
        with open(pdf_path, "wb") as pdf_file:
            pdf_file.write(response_pdf.content)
        print("Downloaded PDF:", pdf_path)
        
        # Read the content of the PDF
        try:
            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                text = ""
                for page_num in range(pdf_reader.numPages):
                    text += pdf_reader.getPage(page_num).extractText()
                
                # Check if the string "Publicación no disponible" is present in the text
                if "Publicación no disponible" in text:
                    os.remove(pdf_path)  # Delete the PDF file
                    print("Deleted PDF:", pdf_path)
        except PyPDF2.utils.PdfReadError:
            print("Failed to read PDF:", pdf_path)
    else:
        print("Failed to retrieve data for PDF link:", pdf_link)


Downloaded PDF: data/pdf/B_1.pdf
Deleted PDF: data/pdf/B_1.pdf
Downloaded PDF: data/pdf/B_2.pdf
Deleted PDF: data/pdf/B_2.pdf
Downloaded PDF: data/pdf/B_3.pdf
Deleted PDF: data/pdf/B_3.pdf
Downloaded PDF: data/pdf/B_4.pdf
Deleted PDF: data/pdf/B_4.pdf
Downloaded PDF: data/pdf/B_5.pdf
Deleted PDF: data/pdf/B_5.pdf
Downloaded PDF: data/pdf/B_6.pdf
Deleted PDF: data/pdf/B_6.pdf
Downloaded PDF: data/pdf/B_7.pdf
Deleted PDF: data/pdf/B_7.pdf
Downloaded PDF: data/pdf/B_8.pdf
Deleted PDF: data/pdf/B_8.pdf
Downloaded PDF: data/pdf/B_9.pdf
Deleted PDF: data/pdf/B_9.pdf
Downloaded PDF: data/pdf/B_10.pdf
Deleted PDF: data/pdf/B_10.pdf
Downloaded PDF: data/pdf/B_11.pdf
Deleted PDF: data/pdf/B_11.pdf
Downloaded PDF: data/pdf/B_12.pdf
Deleted PDF: data/pdf/B_12.pdf
Downloaded PDF: data/pdf/B_13.pdf
Deleted PDF: data/pdf/B_13.pdf
Downloaded PDF: data/pdf/B_14.pdf
Deleted PDF: data/pdf/B_14.pdf
Downloaded PDF: data/pdf/B_15.pdf
Deleted PDF: data/pdf/B_15.pdf
Downloaded PDF: data/pdf/B_16.pdf
Deleted P

# Crear dataframe

Dataframe 4 dígitos

In [95]:
import os
import PyPDF2
import pandas as pd

# Function to read PDF files from a directory
def read_pdfs_from_directory(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Filter out only PDF files
    pdf_files = [file for file in files if file.lower().endswith(".pdf")]
    
    if not pdf_files:
        print("No PDF files found in the directory.")
        return None
    
    # Create an empty list to store DataFrames for each PDF
    pdf_data_list = []
    
    # Iterate over each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(directory, pdf_file)
        print(f"Reading PDF file: {pdf_path}")
        
        # Open the PDF file
        with open(pdf_path, "rb") as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfFileReader(file)
            
            # Get number of pages in the PDF
            num_pages = pdf_reader.numPages
            
            # Initialize text for each page
            pdf_text = ""
            
            # Extract text from each page
            for page_num in range(num_pages):
                page = pdf_reader.getPage(page_num)
                pdf_text += page.extractText()
            
            # Store the text and the last 4 digits of the PDF file name in a DataFrame
            id_file = pdf_file[0:-4]  # Extract the last 4 digits from the file name
            pdf_data_list.append(pd.DataFrame({"Text": [pdf_text], "id_file": [id_file]}))
    
    # Concatenate all DataFrames in the list into a single DataFrame
    pdf_data = pd.concat(pdf_data_list, ignore_index=True)
    
    return pdf_data

# Directory containing PDF files
pdf_directory = "data/pdf"

# Check if the directory exists
if not os.path.exists(pdf_directory):
    print(f"Directory '{pdf_directory}' does not exist.")
else:
    # Read PDFs from the directory
    pdf_dataframe = read_pdfs_from_directory(pdf_directory)
    
    if pdf_dataframe is not None:
        print("\nData from PDFs:")
        print(pdf_dataframe.head())  # Display the first few rows of the DataFrame


Reading PDF file: data/pdf/A_1896.pdf
Reading PDF file: data/pdf/A_1725.pdf
Reading PDF file: data/pdf/A_1084.pdf
Reading PDF file: data/pdf/A_1416.pdf
Reading PDF file: data/pdf/B_435.pdf
Reading PDF file: data/pdf/A_1398.pdf
Reading PDF file: data/pdf/B_745.pdf
Reading PDF file: data/pdf/A_1669.pdf
Reading PDF file: data/pdf/B_774.pdf
Reading PDF file: data/pdf/A_1012.pdf
Reading PDF file: data/pdf/A_824.pdf
Reading PDF file: data/pdf/A_1728.pdf
Reading PDF file: data/pdf/A_660.pdf
Reading PDF file: data/pdf/A_1754.pdf
Reading PDF file: data/pdf/A_1354.pdf
Reading PDF file: data/pdf/A_804.pdf
Reading PDF file: data/pdf/B_754.pdf
Reading PDF file: data/pdf/A_1591.pdf
Reading PDF file: data/pdf/B_963.pdf
Reading PDF file: data/pdf/B_644.pdf
Reading PDF file: data/pdf/A_604.pdf
Reading PDF file: data/pdf/B_399.pdf
Reading PDF file: data/pdf/A_1302.pdf
Reading PDF file: data/pdf/B_734.pdf
Reading PDF file: data/pdf/A_593.pdf
Reading PDF file: data/pdf/A_1724.pdf
Reading PDF file: data/pd

In [145]:
pdf_dataframe[['A', 'B']] = pdf_dataframe["Text"].str.split(': ', n=1, expand=True)
pdf_dataframe[['reporte', 'B']] = pdf_dataframe["B"].str.split('\n', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['fecha_act', 'B']] = pdf_dataframe["B"].str.split('\n', n=1, expand=True)
pdf_dataframe[['nombre', 'B']] = pdf_dataframe["B"].str.split('FECHA DE NACIMIENTO:', n=1, expand=True)
pdf_dataframe[['fecha_nac', 'B']] = pdf_dataframe["B"].str.split('EDAD:', n=1, expand=True)
pdf_dataframe[['edad', 'B']] = pdf_dataframe["B"].str.split('GÉNERO:', n=1, expand=True)
pdf_dataframe[['genero', 'B']] = pdf_dataframe["B"].str.split('FECHA', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['fecha_hechos', 'B']] = pdf_dataframe["B"].str.split('LUGAR', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split(':', n=1, expand=True)
pdf_dataframe[['lugar', 'B']] = pdf_dataframe["B"].str.split('NACIONALIDAD:', n=1, expand=True)
pdf_dataframe[['nacionalidad', 'B']] = pdf_dataframe["B"].str.split('CABELLO:', n=1, expand=True)
pdf_dataframe[['tipo_cabello', 'B']] = pdf_dataframe["B"].str.split('COLOR:', n=1, expand=True)
pdf_dataframe[['color_cabello', 'B']] = pdf_dataframe["B"].str.split('COLOR', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('OJOS:', n=1, expand=True)
pdf_dataframe[['color_ojos', 'B']] = pdf_dataframe["B"].str.split('ESTATURA:', n=1, expand=True)
pdf_dataframe[['estatura', 'B']] = pdf_dataframe["B"].str.split('PESO:', n=1, expand=True)
pdf_dataframe[['peso', 'B']] = pdf_dataframe["B"].str.split('SEÑAS', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('PARTICULARES:', n=1, expand=True)
pdf_dataframe[['senas_part', 'B']] = pdf_dataframe["B"].str.split('RESUMEN', n=1, expand=True)
pdf_dataframe[['A', 'B']] = pdf_dataframe["B"].str.split('HECHOS:', n=1, expand=True)
pdf_dataframe[['resumen_hechos', 'B']] = pdf_dataframe["B"].str.split('RESUMEN', n=1, expand=True)
pdf_dataframe[['senas_part', 'B']] = pdf_dataframe["senas_part"].str.split('Acompañante', n=1, expand=True)
pdf_dataframe[['A', 'acompanante']] = pdf_dataframe["B"].str.split('NOMBRE:', n=1, expand=True)
# pdf_dataframe['id'] = pdf_dataframe['Last_4_Digits'].str.replace(r'_', '', regex=True)
pdf_dataframe['nombre'] = pdf_dataframe['nombre'].str.replace(r'\n', '', regex=True)
pdf_dataframe['edad'] = pdf_dataframe['edad'].str.replace(r'\n', '', regex=True)
pdf_dataframe['fecha_nac'] = pdf_dataframe['fecha_nac'].str.replace(r'\n', '', regex=True)
pdf_dataframe['genero'] = pdf_dataframe['genero'].str.replace(r'\n', '', regex=True)
pdf_dataframe['fecha_hechos'] = pdf_dataframe['fecha_hechos'].str.replace(r'\n', '', regex=True)
pdf_dataframe['lugar'] = pdf_dataframe['lugar'].str.replace(r'\n', '', regex=True)
pdf_dataframe['nacionalidad'] = pdf_dataframe['nacionalidad'].str.replace(r'\n', '', regex=True)
pdf_dataframe['tipo_cabello'] = pdf_dataframe['tipo_cabello'].str.replace(r'\n', '', regex=True)
pdf_dataframe['color_cabello'] = pdf_dataframe['color_cabello'].str.replace(r'\n', '', regex=True)
pdf_dataframe['color_ojos'] = pdf_dataframe['color_ojos'].str.replace(r'\n', '', regex=True)
pdf_dataframe['estatura'] = pdf_dataframe['estatura'].str.replace(r'\n', '', regex=True)
pdf_dataframe['peso'] = pdf_dataframe['peso'].str.replace(r'\n', '', regex=True)
pdf_dataframe['senas_part'] = pdf_dataframe['senas_part'].str.replace(r'\n', '', regex=True)
pdf_dataframe['acompanante'] = pdf_dataframe['acompanante'].str.replace(r'\n', '', regex=True)
pdf_dataframe['resumen_hechos'] = pdf_dataframe['resumen_hechos'].str.replace(r'\n', '', regex=True)
# pdf_dataframe['id'] = 'A' + pdf_dataframe['Last_4_Digits'].astype(str)


df1 = pdf_dataframe[['id_file', 'reporte', 'fecha_act', 'nombre', 'fecha_nac', 'edad', 'genero', 'fecha_hechos',
                    'lugar', 'nacionalidad', 'tipo_cabello', 'color_cabello', 'color_ojos', 'estatura',
                    'peso', 'senas_part', 'resumen_hechos']]

df1 = df1.drop_duplicates(subset=["reporte"], keep='first') 

df1


,id_file,reporte,fecha_act,nombre,fecha_nac,edad,genero,fecha_hechos,lugar,nacionalidad,tipo_cabello,color_cabello,color_ojos,estatura,peso,senas_part,resumen_hechos
0,A_1896,AAMX1896,01/04/2024,ELISA LANDEROS N,05/06/2016,7 años,Femenino,30/03/2024,"TECAMAC, ESTADO DEMEXICO",MEXICANA,Lacio,Castaño_Obscuro,Castaño_Obscuros,1.2 m,22 kg.,NINGUNA.,"EL 30 DE MARZO DE 2024, LA NIÑA ELISA LANDEROS..."
1,A_1725,AAMX1725,02/06/2023,ARIANA ROMINA PÉREZ,11/04/2019,3 años,Femenino,12/06/2022,"CUERNAVACA,MORELOS",MEXICANA,Lacio,Negro,Castaño_Obscuros,1 m,27 kg.,HOYUELOS EN MEJILLA.,"EL 12 DE JUNIO DE 2022, LA NIÑA ARIANA ROMINA ..."
2,A_1084,AAMX1084,14/01/2019,ROSA IRMA LETICIA LÓPEZ,14/05/2014,4 años,Femenino,13/01/2019,"UMAN, YUCATÁN",MEXICANA,Lacio,Castaño_Claro,Castaño_Obscuros,1 m,18 kg.,MANCHA DENACIMIENTO EN ELESTÓMAGO.,"EL 13 DE ENERO DE 2019, LA NIÑA ROSA IRMA LETI..."
3,A_1416,AAMX1416,14/01/2022,JOSELIN GARIBAY MATEOS,26/09/2006,15 años,Femenino,12/01/2022,"ZACATEPEC, MORELOS",MEXICANA,Lacio,Castaño_Obscuro,Castaño_Claros,1.56 m,53 kg.,PADECEESQUIZOFRENIA.,"EL 12 DE ENERO DE 2022, LA ADOLESCENTE JOSELIN..."
4,B_435,AAMX435,11/01/2015,SARA JAKELIN GUZMAN DE LA,21/11/2012,2 años,Femenino,11/01/2015,COAHUILA,MEXICANA,Lacio,Castaño_Claro,Castaño_Obscuros,0.55 m,Sin dato,LUNAR EN UN DEDODEL PIE DERECHO.,"EL 11 DE ENERO DE 2015, LA NIÑA SARA JAKELIN G..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,A_433,AAMX433,08/01/2015,KENIA ITZEL RODRIGUEZ,31/03/1999,15 años,Femenino,27/12/2014,MORELOS,MEXICANA,Ondulado,Castaño_Obscuro,Castaño_Obscuros,1.65 m,68 kg.,CICATRIZ EN LA FRENTEDE APROXIMADAMENTE3 CENT...,EL 27 DE DICIEMBRE DE 2014 LA ADOLESCENTE SE E...
1705,A_1752,AAMX1752,19/07/2023,RENATA SERRATH PÉREZ,29/08/2017,5 años,Femenino,30/06/2023,"CHIGNAHUAPAN,PUEBLA",MEXICANA,Lacio,Negro,Castaño_Obscuros,1 m,Sin dato,LE FALTAN DIENTES ENLA PARTE SUPERIOR.VESTIME...,"EL 30 DE JUNIO DE 2023, LA NIÑA RENATA SERRATH..."
1709,A_1322,AAMX1322,01/06/2021,LAURA JOSELIN DE LA CRUZ,24/02/2006,15 años,Femenino,23/04/2021,"TUXTLA GUTIERREZ,CHIAPAS",MEXICANA,Sin dato,Negro,Castaño_Obscuros,1.53 m,50 kg.,Sin dato,"EL 23 DE ABRIL DE 2021, LA ADOLESCENTE LAURA J..."
1718,A_1720,AAMX1720,31/05/2023,ANDREA RAMOS RIOS,16/08/2009,13 años,Femenino,13/05/2023,"CUAUTITLAN, ESTADODE MEXICO",MEXICANA,Ondulado,Negro,Castaño_Claros,1.56 m,60 kg.,NINGUNA.,"EL 13 DE MAYO DE 2023, LA ADOLESCENTE ANDREA R..."


# Revisar y filtrar qué imágenes queremos antes de descargar

In [146]:
chiapas = df1[df1['lugar'] == "CHIAPAS"]
chiapas

,id_file,reporte,fecha_act,nombre,fecha_nac,edad,genero,fecha_hechos,lugar,nacionalidad,tipo_cabello,color_cabello,color_ojos,estatura,peso,senas_part,resumen_hechos
16,B_754,AAMX754,10/02/2017,LUIS GUSTAVO LOPEZ SANCHEZ,10/11/2016,2 meses,Masculino,06/02/2017,CHIAPAS,MEXICANA,Lacio,Negro,Castaño_Obscuros,0.45 m,6 kg.,LUNAR EN LA ZONA DELACOSTILLA DEL LADODERECHO,"EL 06 DE FEBRERO DE 2017, EL BEBÉ LUIS GUSTAVO..."
466,B_696,AAMX696,18/09/2016,DANIEL EZEQUIEL BALCAZAR,17/07/2016,2 meses,Masculino,16/09/2016,CHIAPAS,MEXICANA,Lacio,Sin_Cabello,Castaño_Obscuros,0.5 m,4 kg.,S/D SospechosoNOMBRE:ADRIANASEÑAS PARTICULARE...,"EL 16 DE SEPTIEMBRE DE 2016, ÉL BEBE DANIEL EZ..."
798,B_828,AAMX828,23/05/2017,SURI DARIANA MOLANO,15/07/2003,13 años,Femenino,20/05/2017,CHIAPAS,MEXICANA,Lacio,Castaño_Claro,Castaño_Obscuros,1.5 m,55 kg.,Sin dato,EL 20 DE MAYO DE 2017 SURI DARIANA MOLANO VILL...
865,A_801,AAMX801,20/04/2017,BRYAN JAVIER RAMIREZ,29/11/2010,6 años,Masculino,21/03/2017,CHIAPAS,MEXICANA,Lacio,Negro,Castaño_Obscuros,1.1 m,19 kg.,CICATRIZ EN LA FRENTECERCA DE LA CEJAIZQUIERDA,"EL NIÑO BRYAN JAVIER RAMIREZ MORALES, FUE VIST..."
1337,A_626,AAMX626,11/03/2016,OSCAR OBED GUILLEN ALBORES,20/03/2015,11 meses,Masculino,10/03/2016,CHIAPAS,MEXICANA,Lacio,Negro,Castaño_Obscuros,0.98 m,9 kg.,Sin dato,"EL 10 DE MARZO DE 2016, EL BEBÉ OSCAR OBED GUI..."


In [147]:
len(chiapas)

5

In [152]:
len(df1)

1261

Crear lista para descargar imágenes

In [149]:
df2 = pd.DataFrame(chiapas["id_file"])
df2[['digitos', 'num']] = df2["id_file"].str.split('_', n=1, expand=True)

df_A = df2[df2['digitos'] == "A"]
df_B = df2[df2['digitos'] == "B"]

list_4_digits = df_A['num'].tolist()
list_3_digits = df_B['num'].tolist()

list_4_digits
list_3_digits

['754', '696', '828']

In [150]:
import os
import requests

# Base URL of the webpage to scrape for images
base_url_image = "https://appalertaamber.fgr.org.mx/Alerta/ObtenerFotoDesaparecido?numero_reporte="

# Function to download images for a list of numbers
def download_images_for_numbers(numbers, images_folder):
    # Create the images folder if it doesn't exist
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    
    # Iterate over the list of numbers
    for number in numbers:
        # Generate the image link for the current number
        image_link = base_url_image + str(number).zfill(3)
        
        # Sending request for image link
        response_image = requests.get(image_link)
        if response_image.status_code == 200:
            # Save the image to the images folder
            image_path = os.path.join(images_folder, f"image_{number}.jpg")
            with open(image_path, "wb") as img_file:
                img_file.write(response_image.content)
            print("Downloaded image:", image_path)
        else:
            print("Failed to retrieve data for image link:", image_link)

# List of numbers
numbers_to_download = list_3_digits

# Folder to save images
images_folder = "data/images"

# Download images for the specified numbers
download_images_for_numbers(numbers_to_download, images_folder)


Downloaded image: data/images/image_754.jpg
Downloaded image: data/images/image_696.jpg
Downloaded image: data/images/image_828.jpg


In [151]:
import os
import requests

# Base URL of the webpage to scrape for images
base_url_image = "https://appalertaamber.fgr.org.mx/Alerta/ObtenerFotoDesaparecido?numero_reporte="

# Function to download images for a list of numbers
def download_images_for_numbers(numbers, images_folder):
    # Create the images folder if it doesn't exist
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    
    # Iterate over the list of numbers
    for number in numbers:
        # Generate the image link for the current number
        image_link = base_url_image + str(number).zfill(4)
        
        # Sending request for image link
        response_image = requests.get(image_link)
        if response_image.status_code == 200:
            # Save the image to the images folder
            image_path = os.path.join(images_folder, f"image_{number}.jpg")
            with open(image_path, "wb") as img_file:
                img_file.write(response_image.content)
            print("Downloaded image:", image_path)
        else:
            print("Failed to retrieve data for image link:", image_link)

# List of numbers
numbers_to_download = list_4_digits

# Folder to save images
images_folder = "data/images"

# Download images for the specified numbers
download_images_for_numbers(numbers_to_download, images_folder)


Downloaded image: data/images/image_801.jpg
Downloaded image: data/images/image_626.jpg


## Transformar

### Revisar archivos vacíos

### Crear dataframe

### Eliminar archivos vacíos

## Cargar

### Subir archivos a S3 o DynamoDB

In [104]:
# import requests

# # Base URLs of the webpages to scrape
# base_url_image = "https://appalertaamber.fgr.org.mx/Alerta/ObtenerFotoDesaparecido?numero_reporte="
# base_url_pdf = "https://appalertaamber.fgr.org.mx/Alerta/CreaAlertaPDFPublico?numero_reporte="

# # Generate links by changing the last 4 digits
# links = [(base_url_image + str(i).zfill(4), base_url_pdf + str(i).zfill(4)) for i in range(1, 10000)]

# # Iterate over the links and send requests to retrieve data
# for index, (image_link, pdf_link) in enumerate(links, start=1):
#     # Sending request for image link
#     response_image = requests.get(image_link)
#     if response_image.status_code == 200:
#         # Save the image to a file
#         with open(f"image_{index}.jpg", "wb") as img_file:
#             img_file.write(response_image.content)
#         print("Downloaded image:", f"image_{index}.jpg")
#     else:
#         print("Failed to retrieve data for image link:", image_link)

#     # Sending request for PDF link
#     response_pdf = requests.get(pdf_link)
#     if response_pdf.status_code == 200:
#         # Save the PDF to a file
#         with open(f"pdf_{index}.pdf", "wb") as pdf_file:
#             pdf_file.write(response_pdf.content)
#         print("Downloaded PDF:", f"pdf_{index}.pdf")
#     else:
#         print("Failed to retrieve data for PDF link:", pdf_link)


In [105]:
# import requests

# # Base URLs of the webpages to scrape
# base_url_image = "https://appalertaamber.fgr.org.mx/Alerta/ObtenerFotoDesaparecido?numero_reporte="
# base_url_pdf = "https://appalertaamber.fgr.org.mx/Alerta/CreaAlertaPDFPublico?numero_reporte="

# # Generate links by changing the last 4 digits
# links = [(base_url_image + str(i).zfill(3), base_url_pdf + str(i).zfill(3)) for i in range(1, 10000)]

# # Iterate over the links and send requests to retrieve data
# for image_link, pdf_link in links:
#     # Sending request for image link
#     response_image = requests.get(image_link)
#     if response_image.status_code == 200:
#         # Do something with the image response, like saving images or processing data
#         # For demonstration, let's print the status code and URL
#         print("Image Status code:", response_image.status_code)
#         print("Image URL:", response_image.url)
#     else:
#         print("Failed to retrieve data for image link:", image_link)

#     # Sending request for PDF link
#     response_pdf = requests.get(pdf_link)
#     if response_pdf.status_code == 200:
#         # Do something with the PDF response, like saving PDFs or processing data
#         # For demonstration, let's print the status code and URL
#         print("PDF Status code:", response_pdf.status_code)
#         print("PDF URL:", response_pdf.url)
#     else:
#         print("Failed to retrieve data for PDF link:", pdf_link)


In [106]:
# soup